# Load the data

## Remember to swap b1 and c2
According to the [class web page](https://github.com/jennybc/stat540_2014/blob/master/homework/hw02/hw02_array-rna-seq-dea.md) from which this normalized data file is drawn, samples b1 and c2 have been swapped.

In [1]:
%%bash
rm GSE37599-data.tsv
wget https://raw.githubusercontent.com/jennybc/stat540_2014/master/examples/yeastPlatforms/data/GSE37599-data.tsv
md5sum GSE37599-data.tsv

rm GSE37599-data.tsv
wget http://www.ugrad.stat.ubc.ca/~stat540/examples/yeastPlatforms/data/GSE37599-data.tsv
md5sum GSE37599-data.tsv
cat <(head -n1 GSE37599-data.tsv) \
<(tail -n +2 GSE37599-data.tsv|gawk 'BEGIN{FS=OFS="\t"}{print $1,$6,$3,$4,$5,$2,$7}') \
> GSE37599-data.unswap.tsv

echo GSE37599-data.tsv
head GSE37599-data.tsv

echo GSE37599-data.unswap.tsv
head GSE37599-data.unswap.tsv

7102fa8268b28f2c185130c0d0218fb4  GSE37599-data.tsv
7102fa8268b28f2c185130c0d0218fb4  GSE37599-data.tsv
GSE37599-data.tsv
""	"b1"	"b2"	"b3"	"c1"	"c2"	"c3"
"1769308_at"	11.1455060343751	6.79559253979714	6.70800306655984	10.9458776869138	6.69886215934826	11.0707252887716
"1769309_at"	2.16432945959427	3.17740803983918	3.12894995537524	2.49830834229269	3.04716851575709	2.44418847894261
"1769310_at"	1.48830007874383	1.42719999067723	1.8156977248348	1.46207537041049	2.07900693485305	1.62303630758998
"1769311_at"	9.00613838230067	9.46182069508223	9.23461767303059	8.97223320966732	9.27670971179903	9.00499688640042
"1769312_at"	6.94571520856451	6.89589516620833	6.95546294237695	6.85145375064933	6.90018668606141	6.89285354827738
"1769313_at"	7.81519169303838	6.60013093986333	6.53441264876916	7.77024503327809	6.56412309047583	7.85177692384053
"1769314_at"	8.6580981215362	8.78837201523564	8.98225957595422	8.57272659379309	8.80755875922876	8.6570395293548
"1769315_at"	1.39561441909922	1.45143141986

--2016-08-26 12:37:43--  https://raw.githubusercontent.com/jennybc/stat540_2014/master/examples/yeastPlatforms/data/GSE37599-data.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.24.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.24.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1252989 (1.2M) [text/plain]
Saving to: ‘GSE37599-data.tsv’

     0K .......... .......... .......... .......... ..........  4% 7.71M 0s
    50K .......... .......... .......... .......... ..........  8% 15.3M 0s
   100K .......... .......... .......... .......... .......... 12% 32.9M 0s
   150K .......... .......... .......... .......... .......... 16% 28.3M 0s
   200K .......... .......... .......... .......... .......... 20% 33.0M 0s
   250K .......... .......... .......... .......... .......... 24% 28.4M 0s
   300K .......... .......... .......... .......... .......... 28% 28.8M 0s
   350K .......... .......... .....

In [2]:
%load_ext rpy2.ipython
import numpy as np
import pandas as pd
import scipy.stats
import statsmodels.sandbox.stats.multicomp

microarray_data_file_path="GSE37599-data.unswap.tsv"

microarray_data=pd.read_table(microarray_data_file_path,index_col=0)

microarray_data.head()

,b1,b2,b3,c1,c2,c3
1769308_at,6.698862,6.795593,6.708003,10.945878,11.145506,11.070725
1769309_at,3.047169,3.177408,3.128950,2.498308,2.164329,2.444188
1769310_at,2.079007,1.427200,1.815698,1.462075,1.488300,1.623036
1769311_at,9.276710,9.461821,9.234618,8.972233,9.006138,9.004997
1769312_at,6.900187,6.895895,6.955463,6.851454,6.945715,6.892854


# Find the mean and standard deviation for all features in the ‘b’ condition and in the ‘c’ condition

In [3]:
b_condition_index=microarray_data.columns[0:3]
c_condition_index=microarray_data.columns[3:6]

In [4]:
b_condition=microarray_data.loc[:,b_condition_index]
b_condition.head()

,b1,b2,b3
1769308_at,6.698862,6.795593,6.708003
1769309_at,3.047169,3.177408,3.128950
1769310_at,2.079007,1.427200,1.815698
1769311_at,9.276710,9.461821,9.234618
1769312_at,6.900187,6.895895,6.955463


In [5]:
c_condition=microarray_data.loc[:,c_condition_index]
c_condition.head()

,c1,c2,c3
1769308_at,10.945878,11.145506,11.070725
1769309_at,2.498308,2.164329,2.444188
1769310_at,1.462075,1.488300,1.623036
1769311_at,8.972233,9.006138,9.004997
1769312_at,6.851454,6.945715,6.892854


In [6]:
b_condition_mean=pd.DataFrame(b_condition.mean(1))
b_condition_mean.columns = ['b_mean']
microarray_data=microarray_data.merge(b_condition_mean,left_index=True,right_index=True)
b_condition_mean.head()

,b_mean
1769308_at,6.734153
1769309_at,3.117842
1769310_at,1.773968
1769311_at,9.324383
1769312_at,6.917182


In [7]:
b_condition_std=pd.DataFrame(b_condition.std(1))
b_condition_std.columns = ['b_std']
microarray_data=microarray_data.merge(b_condition_std,left_index=True,right_index=True)
b_condition_std.head()

,b_std
1769308_at,0.053404
1769309_at,0.065826
1769310_at,0.327901
1769311_at,0.120871
1769312_at,0.033222


In [8]:
c_condition_mean=pd.DataFrame(c_condition.mean(1))
c_condition_mean.columns = ['c_mean']
microarray_data=microarray_data.merge(c_condition_mean,left_index=True,right_index=True)
c_condition_mean.head()

,c_mean
1769308_at,11.054036
1769309_at,2.368942
1769310_at,1.524471
1769311_at,8.994456
1769312_at,6.896674


In [9]:
c_condition_std=pd.DataFrame(c_condition.std(1))
c_condition_std.columns = ['c_std']
microarray_data=microarray_data.merge(c_condition_std,left_index=True,right_index=True)
c_condition_std.head()

,c_std
1769308_at,0.100855
1769309_at,0.179254
1769310_at,0.086362
1769311_at,0.019254
1769312_at,0.047247


In [10]:
microarray_data.head()

,b1,b2,b3,c1,c2,c3,b_mean,b_std,c_mean,c_std
1769308_at,6.698862,6.795593,6.708003,10.945878,11.145506,11.070725,6.734153,0.053404,11.054036,0.100855
1769309_at,3.047169,3.177408,3.128950,2.498308,2.164329,2.444188,3.117842,0.065826,2.368942,0.179254
1769310_at,2.079007,1.427200,1.815698,1.462075,1.488300,1.623036,1.773968,0.327901,1.524471,0.086362
1769311_at,9.276710,9.461821,9.234618,8.972233,9.006138,9.004997,9.324383,0.120871,8.994456,0.019254
1769312_at,6.900187,6.895895,6.955463,6.851454,6.945715,6.892854,6.917182,0.033222,6.896674,0.047247


# Find the fold change between the ‘b’ condition and the ‘c’ condition for all features (hint: use your calculated means for this).

In [11]:
fold_change=pd.DataFrame(b_condition_mean['b_mean']/c_condition_mean['c_mean'])
fold_change.columns=['fold_change']
log2_fold_change=np.log2(fold_change)
log2_fold_change.columns=['log2_fold_change']
abs_log2_fold_change=np.abs(log2_fold_change)
abs_log2_fold_change.columns=['abs_log2_fold_change']
microarray_data=microarray_data.merge(fold_change,left_index=True,right_index=True)
microarray_data=microarray_data.merge(log2_fold_change,left_index=True,right_index=True)
microarray_data=microarray_data.merge(abs_log2_fold_change,left_index=True,right_index=True)
microarray_data.head()

,b1,b2,b3,c1,c2,c3,b_mean,b_std,c_mean,c_std,fold_change,log2_fold_change,abs_log2_fold_change
1769308_at,6.698862,6.795593,6.708003,10.945878,11.145506,11.070725,6.734153,0.053404,11.054036,0.100855,0.609203,-0.715005,0.715005
1769309_at,3.047169,3.177408,3.128950,2.498308,2.164329,2.444188,3.117842,0.065826,2.368942,0.179254,1.316133,0.396305,0.396305
1769310_at,2.079007,1.427200,1.815698,1.462075,1.488300,1.623036,1.773968,0.327901,1.524471,0.086362,1.163662,0.218672,0.218672
1769311_at,9.276710,9.461821,9.234618,8.972233,9.006138,9.004997,9.324383,0.120871,8.994456,0.019254,1.036681,0.051972,0.051972
1769312_at,6.900187,6.895895,6.955463,6.851454,6.945715,6.892854,6.917182,0.033222,6.896674,0.047247,1.002974,0.004284,0.004284


# How many of these features are statistically significantly different (p < 0.05) from each other between the ‘b’ and the ‘c’ condition?

In [12]:
p_value=pd.DataFrame(scipy.stats.ttest_ind(b_condition,c_condition,axis=1).pvalue)
p_value.columns=['p_value']
p_value.index=microarray_data.index
microarray_data=microarray_data.merge(p_value,left_index=True,right_index=True)
p_value.head()

,p_value
1769308_at,3.242072e-07
1769309_at,2.452944e-03
1769310_at,2.715102e-01
1769311_at,9.526176e-03
1769312_at,5.718298e-01


In [13]:
microarray_data.head()

,b1,b2,b3,c1,c2,c3,b_mean,b_std,c_mean,c_std,fold_change,log2_fold_change,abs_log2_fold_change,p_value
1769308_at,6.698862,6.795593,6.708003,10.945878,11.145506,11.070725,6.734153,0.053404,11.054036,0.100855,0.609203,-0.715005,0.715005,3.242072e-07
1769309_at,3.047169,3.177408,3.128950,2.498308,2.164329,2.444188,3.117842,0.065826,2.368942,0.179254,1.316133,0.396305,0.396305,2.452944e-03
1769310_at,2.079007,1.427200,1.815698,1.462075,1.488300,1.623036,1.773968,0.327901,1.524471,0.086362,1.163662,0.218672,0.218672,2.715102e-01
1769311_at,9.276710,9.461821,9.234618,8.972233,9.006138,9.004997,9.324383,0.120871,8.994456,0.019254,1.036681,0.051972,0.051972,9.526176e-03
1769312_at,6.900187,6.895895,6.955463,6.851454,6.945715,6.892854,6.917182,0.033222,6.896674,0.047247,1.002974,0.004284,0.004284,5.718298e-01


In [14]:
microarray_data.tail()

,b1,b2,b3,c1,c2,c3,b_mean,b_std,c_mean,c_std,fold_change,log2_fold_change,abs_log2_fold_change,p_value
RPTR-Sc-U89963-1_s_at,1.261361,1.193256,1.099165,1.356363,1.394847,1.419373,1.184594,0.081444,1.390195,0.031762,0.852107,-0.230894,0.230894,0.015177
RPTR-Sc-X03453-1_at,1.142379,1.656462,1.027002,1.423083,1.169068,1.100052,1.275281,0.335115,1.230734,0.170115,1.036195,0.051296,0.051296,0.847360
RPTR-Sc-X58791-1_at,0.989840,1.240098,1.270160,1.129246,1.240098,1.387374,1.166699,0.153901,1.252239,0.129492,0.931691,-0.102077,0.102077,0.502214
RPTR-Sc-X58791-1_s_at,1.121985,1.142665,1.172726,1.172762,1.165211,1.068312,1.145792,0.025515,1.135428,0.058247,1.009127,0.013108,0.013108,0.791733
RPTR-Sc-X58791-2_s_at,2.651024,2.877677,2.651024,2.525995,2.602404,2.508159,2.726575,0.130858,2.545519,0.050064,1.071127,0.099130,0.099130,0.088794


In [15]:
microarray_data_signficant_pval = microarray_data[microarray_data.p_value < 0.05]
microarray_data_signficant_pval.head()

,b1,b2,b3,c1,c2,c3,b_mean,b_std,c_mean,c_std,fold_change,log2_fold_change,abs_log2_fold_change,p_value
1769308_at,6.698862,6.795593,6.708003,10.945878,11.145506,11.070725,6.734153,0.053404,11.054036,0.100855,0.609203,-0.715005,0.715005,3.242072e-07
1769309_at,3.047169,3.177408,3.128950,2.498308,2.164329,2.444188,3.117842,0.065826,2.368942,0.179254,1.316133,0.396305,0.396305,2.452944e-03
1769311_at,9.276710,9.461821,9.234618,8.972233,9.006138,9.004997,9.324383,0.120871,8.994456,0.019254,1.036681,0.051972,0.051972,9.526176e-03
1769313_at,6.564123,6.600131,6.534413,7.770245,7.815192,7.851777,6.566222,0.032909,7.812405,0.040837,0.840487,-0.250703,0.250703,2.083371e-06
1769314_at,8.807559,8.788372,8.982260,8.572727,8.658098,8.657040,8.859397,0.106834,8.629288,0.048987,1.026666,0.037967,0.037967,2.750078e-02


In [16]:
number_of_statistically_significant_features_by_pval = microarray_data_signficant_pval.shape[0]
print("There are %d statistically significant features with p value less than 0.05." % number_of_statistically_significant_features_by_pval)

There are 5028 statistically significant features with p value less than 0.05.


# What are the top ten probe IDs with the most extreme fold change while still maintaining statistical significance?

In [17]:
microarray_data_signficant_pval_sort_by_abs_log2_fold_change = microarray_data_signficant_pval.sort_values(['abs_log2_fold_change'],ascending=False)
microarray_data_signficant_pval_sort_by_abs_log2_fold_change_top_10=microarray_data_signficant_pval_sort_by_abs_log2_fold_change.head(10)
microarray_data_signficant_pval_sort_by_abs_log2_fold_change_top_10

,b1,b2,b3,c1,c2,c3,b_mean,b_std,c_mean,c_std,fold_change,log2_fold_change,abs_log2_fold_change,p_value
1772391_at,2.109841,1.904828,1.949609,12.199319,12.149454,11.949915,1.988093,0.107788,12.099563,0.131975,0.164311,-2.605498,2.605498,5.373301e-08
1776153_at,1.730429,1.846854,1.749672,9.183171,9.189375,8.867755,1.775652,0.062409,9.080101,0.183923,0.195554,-2.354360,2.354360,3.327179e-07
1772361_at,1.655549,1.482476,1.409780,7.154393,7.633577,7.690351,1.515935,0.126254,7.492774,0.294418,0.202320,-2.305292,2.305292,5.466830e-06
1773702_at,2.251155,1.870127,2.165951,9.858925,10.559120,10.545720,2.095744,0.199981,10.321255,0.400446,0.203051,-2.300084,2.300084,5.807391e-06
1776778_at,2.537141,1.927257,1.946434,9.625571,9.757991,9.752772,2.136944,0.346713,9.712111,0.074991,0.220029,-2.184236,2.184236,3.190218e-06
1770587_at,2.657840,2.092714,2.321151,10.893532,10.487541,10.322389,2.357235,0.284286,10.567821,0.293912,0.223058,-2.164511,2.164511,4.078591e-06
1774072_at,2.790547,2.595897,2.407478,11.361570,11.054525,11.110367,2.597974,0.191543,11.175487,0.163553,0.232471,-2.104879,2.104879,4.946906e-07
1774122_at,2.670929,2.574048,2.601647,10.924645,11.260103,11.083358,2.615541,0.049912,11.089369,0.167810,0.235860,-2.083996,2.083996,1.213611e-07
1775070_at,2.519148,2.310268,2.011715,9.650794,9.668148,9.531641,2.280377,0.255034,9.616861,0.074311,0.237123,-2.076294,2.076294,1.142515e-06
1769342_at,2.779985,2.019523,2.783414,10.348219,10.563726,10.448946,2.527641,0.440046,10.453630,0.107830,0.241796,-2.048141,2.048141,7.066266e-06


# If you applied a correction for family-wise error rate to your analysis (e.g. a Bonferroni correction), would your top probe IDs still be significant?

In [18]:
fwer=pd.DataFrame(statsmodels.sandbox.stats.multicomp.multipletests(p_value['p_value'],method='bonferroni')[1])
fwer.columns=['fwer']
fwer.index=microarray_data.index
microarray_data=microarray_data.merge(fwer,left_index=True,right_index=True)
fwer.head()

,fwer
1769308_at,0.003543
1769309_at,1.000000
1769310_at,1.000000
1769311_at,1.000000
1769312_at,1.000000


In [19]:
microarray_data.loc[microarray_data_signficant_pval_sort_by_abs_log2_fold_change_top_10.index.values,:]

,b1,b2,b3,c1,c2,c3,b_mean,b_std,c_mean,c_std,fold_change,log2_fold_change,abs_log2_fold_change,p_value,fwer
1772391_at,2.109841,1.904828,1.949609,12.199319,12.149454,11.949915,1.988093,0.107788,12.099563,0.131975,0.164311,-2.605498,2.605498,5.373301e-08,0.000587
1776153_at,1.730429,1.846854,1.749672,9.183171,9.189375,8.867755,1.775652,0.062409,9.080101,0.183923,0.195554,-2.354360,2.354360,3.327179e-07,0.003636
1772361_at,1.655549,1.482476,1.409780,7.154393,7.633577,7.690351,1.515935,0.126254,7.492774,0.294418,0.202320,-2.305292,2.305292,5.466830e-06,0.059742
1773702_at,2.251155,1.870127,2.165951,9.858925,10.559120,10.545720,2.095744,0.199981,10.321255,0.400446,0.203051,-2.300084,2.300084,5.807391e-06,0.063463
1776778_at,2.537141,1.927257,1.946434,9.625571,9.757991,9.752772,2.136944,0.346713,9.712111,0.074991,0.220029,-2.184236,2.184236,3.190218e-06,0.034863
1770587_at,2.657840,2.092714,2.321151,10.893532,10.487541,10.322389,2.357235,0.284286,10.567821,0.293912,0.223058,-2.164511,2.164511,4.078591e-06,0.044571
1774072_at,2.790547,2.595897,2.407478,11.361570,11.054525,11.110367,2.597974,0.191543,11.175487,0.163553,0.232471,-2.104879,2.104879,4.946906e-07,0.005406
1774122_at,2.670929,2.574048,2.601647,10.924645,11.260103,11.083358,2.615541,0.049912,11.089369,0.167810,0.235860,-2.083996,2.083996,1.213611e-07,0.001326
1775070_at,2.519148,2.310268,2.011715,9.650794,9.668148,9.531641,2.280377,0.255034,9.616861,0.074311,0.237123,-2.076294,2.076294,1.142515e-06,0.012485
1769342_at,2.779985,2.019523,2.783414,10.348219,10.563726,10.448946,2.527641,0.440046,10.453630,0.107830,0.241796,-2.048141,2.048141,7.066266e-06,0.077220


## Verify against the implementation in R stats p.adjust

In [20]:
microarray_data_output_file_path='microarray_data.r_input.txt'
microarray_data.to_csv(microarray_data_output_file_path, sep='\t', index=True, index_label='probe_id')

In [21]:
%%bash
head microarray_data.r_input.txt|column -t

probe_id      b1                 b2                  b3                  c1                  c2                  c3                  b_mean              b_std                 c_mean              c_std                 fold_change         log2_fold_change      abs_log2_fold_change  p_value                 fwer
1769308_at    6.69886215934826   6.7955925397971395  6.708003066559839   10.9458776869138    11.1455060343751    11.0707252887716    6.7341525885684135  0.05340449186542181   11.054036336686833  0.10085514423567127   0.6092030443412496  -0.7150049435331856   0.7150049435331856    3.2420722365753934e-07  0.00354293654012959
1769309_at    3.04716851575709   3.17740803983918    3.1289499553752402  2.49830834229269    2.16432945959427    2.4441884789426096  3.117842170323837   0.06582644282386427   2.368942093609857   0.17925397457617243   1.3161327069724977  0.3963049647862665    0.3963049647862665    0.002452943800030417    1.0
1769310_at    2.07900693485305   1.4271999906772301  1.8

In [22]:
%%R
library(stats)

microarray_data_input_file_path='microarray_data.r_input.txt'
microarray_data_output_file_path='microarray_data.r_output.txt'
microarray_data=read.delim(microarray_data_input_file_path,row.names=1)
head(microarray_data)

                 b1       b2       b3        c1        c2        c3   b_mean
1769308_at 6.698862 6.795593 6.708003 10.945878 11.145506 11.070725 6.734153
1769309_at 3.047169 3.177408 3.128950  2.498308  2.164329  2.444188 3.117842
1769310_at 2.079007 1.427200 1.815698  1.462075  1.488300  1.623036 1.773968
1769311_at 9.276710 9.461821 9.234618  8.972233  9.006138  9.004997 9.324383
1769312_at 6.900187 6.895895 6.955463  6.851454  6.945715  6.892854 6.917182
1769313_at 6.564123 6.600131 6.534413  7.770245  7.815192  7.851777 6.566222
                b_std    c_mean      c_std fold_change log2_fold_change
1769308_at 0.05340449 11.054036 0.10085514   0.6092030     -0.715004944
1769309_at 0.06582644  2.368942 0.17925397   1.3161327      0.396304965
1769310_at 0.32790103  1.524471 0.08636165   1.1636618      0.218671848
1769311_at 0.12087116  8.994456 0.01925410   1.0366811      0.051972164
1769312_at 0.03322198  6.896674 0.04724673   1.0029735      0.004283524
1769313_at 0.03290939  7.8124

In [23]:
%%R
tail(microarray_data)

                            b1       b2       b3       c1       c2       c3
RPTR-Sc-U89963-1_at   1.090372 1.116685 1.254244 1.152728 1.119306 1.039859
RPTR-Sc-U89963-1_s_at 1.261361 1.193256 1.099165 1.356363 1.394847 1.419373
RPTR-Sc-X03453-1_at   1.142379 1.656462 1.027002 1.423083 1.169068 1.100052
RPTR-Sc-X58791-1_at   0.989840 1.240098 1.270160 1.129246 1.240098 1.387374
RPTR-Sc-X58791-1_s_at 1.121985 1.142665 1.172726 1.172762 1.165211 1.068312
RPTR-Sc-X58791-2_s_at 2.651024 2.877677 2.651024 2.525995 2.602404 2.508159
                        b_mean      b_std   c_mean      c_std fold_change
RPTR-Sc-U89963-1_at   1.153767 0.08800469 1.103964 0.05797714   1.0451126
RPTR-Sc-U89963-1_s_at 1.184594 0.08144401 1.390195 0.03176165   0.8521065
RPTR-Sc-X03453-1_at   1.275281 0.33511488 1.230734 0.17011531   1.0361952
RPTR-Sc-X58791-1_at   1.166699 0.15390051 1.252239 0.12949154   0.9316905
RPTR-Sc-X58791-1_s_at 1.145792 0.02551495 1.135428 0.05824695   1.0091274
RPTR-Sc-X58791-2_s_at 2.

In [24]:
%%R
microarray_data$FWER = 1
microarray_data$FWER <- p.adjust(microarray_data$p_value,'bonferroni')
head(microarray_data)

                 b1       b2       b3        c1        c2        c3   b_mean
1769308_at 6.698862 6.795593 6.708003 10.945878 11.145506 11.070725 6.734153
1769309_at 3.047169 3.177408 3.128950  2.498308  2.164329  2.444188 3.117842
1769310_at 2.079007 1.427200 1.815698  1.462075  1.488300  1.623036 1.773968
1769311_at 9.276710 9.461821 9.234618  8.972233  9.006138  9.004997 9.324383
1769312_at 6.900187 6.895895 6.955463  6.851454  6.945715  6.892854 6.917182
1769313_at 6.564123 6.600131 6.534413  7.770245  7.815192  7.851777 6.566222
                b_std    c_mean      c_std fold_change log2_fold_change
1769308_at 0.05340449 11.054036 0.10085514   0.6092030     -0.715004944
1769309_at 0.06582644  2.368942 0.17925397   1.3161327      0.396304965
1769310_at 0.32790103  1.524471 0.08636165   1.1636618      0.218671848
1769311_at 0.12087116  8.994456 0.01925410   1.0366811      0.051972164
1769312_at 0.03322198  6.896674 0.04724673   1.0029735      0.004283524
1769313_at 0.03290939  7.8124

In [25]:
%%R
write.table(microarray_data, file = microarray_data_output_file_path, 
            append = FALSE, quote = FALSE, sep = "\t",
            eol = "\n", na = "NA", dec = ".", row.names = TRUE, 
            col.names = NA, qmethod = c("escape", "double"),
            fileEncoding = "")

In [26]:
%%bash
head microarray_data.r_output.txt|column -t

b1            b2                b3                c1                c2                c3                b_mean            b_std             c_mean              c_std             fold_change         log2_fold_change   abs_log2_fold_change  p_value              fwer                  FWER
1769308_at    6.69886215934826  6.79559253979714  6.70800306655984  10.9458776869138  11.1455060343751  11.0707252887716  6.73415258856841  0.0534044918654218  11.0540363366868  0.100855144235671   0.60920304434125   -0.715004943533186    0.715004943533186    3.24207223657539e-07  0.00354293654012959  0.00354293654012959
1769309_at    3.04716851575709  3.17740803983918  3.12894995537524  2.49830834229269  2.16432945959427  2.44418847894261  3.11784217032384  0.0658264428238643  2.36894209360986  0.179253974576172   1.3161327069725    0.396304964786266     0.396304964786266    0.00245294380003042   1                    1
1769310_at    2.07900693485305  1.42719999067723  1.8156977248348   1.46207537041049 

In [27]:
microarray_data_r_output_file_path='microarray_data.r_output.txt'

microarray_data=pd.read_table(microarray_data_r_output_file_path,index_col=0)

microarray_data.head()

,b1,b2,b3,c1,c2,c3,b_mean,b_std,c_mean,c_std,fold_change,log2_fold_change,abs_log2_fold_change,p_value,fwer,FWER
1769308_at,6.698862,6.795593,6.708003,10.945878,11.145506,11.070725,6.734153,0.053404,11.054036,0.100855,0.609203,-0.715005,0.715005,3.242072e-07,0.003543,0.003543
1769309_at,3.047169,3.177408,3.128950,2.498308,2.164329,2.444188,3.117842,0.065826,2.368942,0.179254,1.316133,0.396305,0.396305,2.452944e-03,1.000000,1.000000
1769310_at,2.079007,1.427200,1.815698,1.462075,1.488300,1.623036,1.773968,0.327901,1.524471,0.086362,1.163662,0.218672,0.218672,2.715102e-01,1.000000,1.000000
1769311_at,9.276710,9.461821,9.234618,8.972233,9.006138,9.004997,9.324383,0.120871,8.994456,0.019254,1.036681,0.051972,0.051972,9.526176e-03,1.000000,1.000000
1769312_at,6.900187,6.895895,6.955463,6.851454,6.945715,6.892854,6.917182,0.033222,6.896674,0.047247,1.002974,0.004284,0.004284,5.718298e-01,1.000000,1.000000


In [28]:
microarray_data_signficant_pval_sort_by_abs_log2_fold_change_top_10_with_fwer=microarray_data.loc[microarray_data_signficant_pval_sort_by_abs_log2_fold_change_top_10.index.values,:]
microarray_data_signficant_pval_sort_by_abs_log2_fold_change_top_10_with_fwer

,b1,b2,b3,c1,c2,c3,b_mean,b_std,c_mean,c_std,fold_change,log2_fold_change,abs_log2_fold_change,p_value,fwer,FWER
1772391_at,2.109841,1.904828,1.949609,12.199319,12.149454,11.949915,1.988093,0.107788,12.099563,0.131975,0.164311,-2.605498,2.605498,5.373301e-08,0.000587,0.000587
1776153_at,1.730429,1.846854,1.749672,9.183171,9.189375,8.867755,1.775652,0.062409,9.080101,0.183923,0.195554,-2.354360,2.354360,3.327179e-07,0.003636,0.003636
1772361_at,1.655549,1.482476,1.409780,7.154393,7.633577,7.690351,1.515935,0.126254,7.492774,0.294418,0.202320,-2.305292,2.305292,5.466830e-06,0.059742,0.059742
1773702_at,2.251155,1.870127,2.165951,9.858925,10.559120,10.545720,2.095744,0.199981,10.321255,0.400446,0.203051,-2.300084,2.300084,5.807391e-06,0.063463,0.063463
1776778_at,2.537141,1.927257,1.946434,9.625571,9.757991,9.752772,2.136944,0.346713,9.712111,0.074991,0.220029,-2.184236,2.184236,3.190218e-06,0.034863,0.034863
1770587_at,2.657840,2.092714,2.321151,10.893532,10.487541,10.322389,2.357235,0.284286,10.567821,0.293912,0.223058,-2.164511,2.164511,4.078591e-06,0.044571,0.044571
1774072_at,2.790547,2.595897,2.407478,11.361570,11.054525,11.110367,2.597974,0.191543,11.175487,0.163553,0.232471,-2.104879,2.104879,4.946906e-07,0.005406,0.005406
1774122_at,2.670929,2.574048,2.601647,10.924645,11.260103,11.083358,2.615541,0.049912,11.089369,0.167810,0.235860,-2.083996,2.083996,1.213611e-07,0.001326,0.001326
1775070_at,2.519148,2.310268,2.011715,9.650794,9.668148,9.531641,2.280377,0.255034,9.616861,0.074311,0.237123,-2.076294,2.076294,1.142515e-06,0.012485,0.012485
1769342_at,2.779985,2.019523,2.783414,10.348219,10.563726,10.448946,2.527641,0.440046,10.453630,0.107830,0.241796,-2.048141,2.048141,7.066266e-06,0.077220,0.077220


In [29]:
number_of_statistically_significant_features_by_fwer = microarray_data_signficant_pval_sort_by_abs_log2_fold_change_top_10_with_fwer[microarray_data_signficant_pval_sort_by_abs_log2_fold_change_top_10_with_fwer.fwer<0.05].shape[0]
print("There are %d statistically significant features with fwer less than 0.05 from the original top 10." % number_of_statistically_significant_features_by_fwer)

There are 7 statistically significant features with fwer less than 0.05 from the original top 10.


# You can find the names of the genes of your top ten probe IDs

In [30]:
print("\n".join(microarray_data_signficant_pval_sort_by_abs_log2_fold_change_top_10_with_fwer.index.values))

1772391_at
1776153_at
1772361_at
1773702_at
1776778_at
1770587_at
1774072_at
1774122_at
1775070_at
1769342_at


## What do these results suggest is happening differently between batch and chemostat in the cell?
Results from DAVID suggest that "energy derivation by oxidation of organic compounds" and "generation of precursor metabolites and energy" are happening differently between batch and chemostat conditions. This is consistent with the study design.

> In our study, we sought to evaluate different processing methods, as well as statistical methods to identify DGE and cross-compare the results with microarray analysis; for this purpose, we used as case study the transcriptome of *S. cerevisiae* laboratory strain CEN.PK 113-7D under two different metabolic conditions: respiro-fermentative (batch) or fully respiratory (chemostat) metabolism.
-- http://nar.oxfordjournals.org/content/40/20/10084.full

## What is the meaning of the p-values listed next to the terms in the functional annotation chart?
These p-values are less than 0.05, indicating that these gene ontologies are statistically significant by DAVID's modified Fisher's exact test.